In [23]:
import snap
import random
import numpy
S = 10**4
TPATH = "/courses/TSKS33/ht2023/data/"
def genmod10star():   
    #G1 = snap.GenPrefAttach(1000, 10)
    #G1 = snap.GenRndPowerLaw(100000,2.2)
    G = snap.GenStar(snap.PUNGraph,10,False)
    G.AddEdge(4,5)
    #G = snap.GetMxScc(G1)
    N = G.GetNodes()
    
    # node numbering in the mat file is sequential n=1,2,... following the node iterator
    # gio.h line 369
#    snap.SaveMatlabSparseMtx(G, "mod10star.mat")

    # assign the degree as attribute
    x = snap.TIntFltH(N)
#    file = open("mod10star_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        x[n]=NI.GetDeg()
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
#    file.close()
    
    return G, x

def genLiveJournal():   
    G1 = snap.LoadEdgeList(snap.PUNGraph, TPATH + "soc-LiveJournal1.txt", 0, 1)
    snap.DelSelfEdges(G1)
    G = snap.GetMxScc(G1)
#    print(snap.IsConnected(G))
    N = G.GetNodes()

#    snap.SaveMatlabSparseMtx(G, "LiveJournal.mat")

    # assign synthetic data as attributes
    x = snap.TIntFltH(N)
#    file = open("LiveJournal_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        k=NI.GetDeg()
        x[n]=50000*(1+k/10.+0.1*numpy.sin(n))
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
 
#    file.close()
    return G, x


In [25]:
G, x = genLiveJournal()

KeyboardInterrupt: 

random connection of random node: 598522.277


In [40]:

sum = 0
for n in G.Nodes():
    t = x[n.GetId()]
    sum+=t
exact = sum/G.GetNodes()
print("-- expected values of <x>-hat -----")
print("uniform sampling:", exact)

sum = 0
for i in G.Nodes():
    node = G.GetNI(i.GetId())
    tmp = 0
    for nbr_id in node.GetOutEdges():
        nbr = G.GetNI(nbr_id)
        t = x[nbr.GetId()]
        tmp+=t
    sum += (tmp / node.GetDeg())

print("random connection of random node:", round(sum/G.GetNodes(), 3))

sum = 0

for node in G.Nodes():
    for nbr_id in node.GetOutEdges():
        nbr = G.GetNI(nbr_id)
        sum += x[nbr.GetId()]
    
print("uniform random walk:", sum/(2*G.GetEdges()))

sum = 0
for n in G.Nodes():
    t = x[n.GetId()]
    sum+=t
print("M-H random walk:", sum/G.GetNodes())

-- expected values of <x>-hat -----
uniform sampling: 138451.89439110097
random connection of random node: 598522.277
uniform random walk: 903319.3950024237
M-H random walk: 138451.89439110097


In [32]:
num_runs = 5
for i in range(num_runs):
    tmp_sum = 0
    Rnd = snap.TRnd(i)
    Rnd.Randomize()
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        tmp_sum += x[nid]
    print("uniform sampling:", tmp_sum/S)


uniform sampling: 138683.33896403696
uniform sampling: 138559.0876252566
uniform sampling: 133593.49784777864
uniform sampling: 138557.16823338994
uniform sampling: 141212.32718854563


In [34]:
import random
for i in range(num_runs):
    sum = 0
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        #nbr = G.GetNI(nbr)
        sum+=x[nbr]

    print("random connection of random node:", sum/S)

random connection of random node: 601513.5820804201
random connection of random node: 595166.2953817098
random connection of random node: 667870.9590805494
random connection of random node: 641500.359067879
random connection of random node: 624017.7049644771


In [36]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        node = G.GetNI(nbr)
        sum += x[nbr]
    
    print("uniform random walk:", sum/S)

uniform random walk: 902410.230411075
uniform random walk: 795069.0669079883
uniform random walk: 966658.1215538473
uniform random walk: 1227003.7886536017
uniform random walk: 765698.9496282376


In [37]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)

    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        nbr = G.GetNI(nbr)
        nbr_deg = nbr.GetDeg()
        node_deg = node.GetDeg()
        sum += x[node.GetId()]
        if random.uniform(0, 1) < (node_deg/nbr_deg):
            node = nbr
    print("uniform random walk:", sum/S)

uniform random walk: 134065.4942059566
uniform random walk: 138064.68589515277
uniform random walk: 135464.13079592356
uniform random walk: 129501.81654892623
uniform random walk: 129942.15457139573
